# Goal

This notebook demonstrates how to calculate the shortest path from a given point to all possible nodes in a road network using GraphX in a distributed manner. The workflow leverages Databricks, Scala and SQL, and GraphX to efficiently compute shortest paths at scale.

In [0]:
%run ../get_user

In [0]:
user_email = spark.sql("SELECT current_user()").collect()[0][0]
username = get_username_from_email(user_email)
dbutils.widgets.text("username", username)
print(username)

In [0]:
%scala
import org.apache.sedona.spark.SedonaContext


val config = SedonaContext.builder()
.appName("graphs") // Change this to a proper name
.getOrCreate()
val sedona = SedonaContext.create(config)

In [0]:
%scala
val username = dbutils.widgets.get("username")

In [0]:
%scala
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.graphx.{Graph, Edge}
import org.apache.spark.rdd.RDD
import scala.reflect.ClassTag
import org.apache.spark.graphx._
import scala.reflect.ClassTag
import org.apache.spark.graphx._
import org.apache.spark.sql.functions._
import spark.implicits._

In [0]:
# %scala
# // Create a SparkSession
# val spark = SparkSession.builder().appName("Graphs").getOrCreate()

In [0]:
%scala
val londonArea = spark.sql(s"""
  SELECT b.name, ST_Buffer(b.geometry, 5) AS geometry
  FROM geospatial.lookups.boundary_line_ceremonial_counties_$username b
  WHERE b.name IN ('City and County of the City of London')
""")

londonArea.createOrReplaceTempView("london_area_vw")

In [0]:
%scala
val londonArea = spark.sql(s"""
  SELECT b.name, ST_Buffer(b.geometry, 5) AS geometry
  FROM geospatial.lookups.boundary_line_ceremonial_counties_$username b 
  WHERE b.name IN ('City and County of the City of London')
""")

londonArea.createOrReplaceTempView("london_area_vw")

In [0]:
%scala
val londonRoadNodes = spark.sql(s"""
  SELECT a.fid, a.id, a.form_of_road_node, a.geometry
  FROM geospatial.networks.road_node_$username a
  INNER JOIN london_area_vw b 
    ON st_within(a.geometry, b.geometry)
""")
londonRoadNodes.createOrReplaceTempView("london_road_nodes_vw")

In [0]:
%scala
// This code selects all road edges within the City of London area by joining road links with road nodes,
// ensuring both start and end nodes are present in the area. It creates a DataFrame with edges in both directions
// and registers it as a temporary view for further graph processing.

val londonRoadEdges = spark.sql(s"""
  SELECT a.fid, a.id, b.fid AS src, c.fid AS dst, a.length AS distance
  FROM geospatial.networks.road_link_$username a
  LEFT JOIN london_road_nodes_vw b
    ON a.start_node = b.id
  LEFT JOIN london_road_nodes_vw c
    ON a.end_node = c.id
  WHERE b.id IS NOT NULL
    AND c.id IS NOT NULL
  UNION ALL
  SELECT a.fid, a.id, c.fid AS src, b.fid AS dst, a.length AS distance
  FROM geospatial.networks.road_link_$username a
  LEFT JOIN london_road_nodes_vw b
    ON a.start_node = b.id
  LEFT JOIN london_road_nodes_vw c
    ON a.end_node = c.id
  WHERE b.id IS NOT NULL
    AND c.id IS NOT NULL
""")
londonRoadEdges.createOrReplaceTempView("london_road_edges_vw")

In [0]:
%scala
// This block constructs a GraphX graph from London road network nodes and edges.
// Vertices are defined as (id, name) and edges as (src, dst, distance).

// Define vertices: (id, name)
val vertexDF = londonRoadNodes.select(
  col("fid").cast("long").alias("id"),
  col("id").alias("name")
)

// Define edges: (src, dst, distance)
val edgeDF = londonRoadEdges.select(
  col("src").cast("long").alias("src"),
  col("dst").cast("long").alias("dst"),
  col("distance")
)

val vertexRDD = vertexDF.rdd.map(row => (row.getAs[Long]("id"), row.getAs[String]("name")))
val edgeRDD = edgeDF.rdd.map(row => 
  Edge(row.getAs[Long]("src"), row.getAs[Long]("dst"), row.getAs[Double]("distance"))
)

val graph = Graph(vertexRDD, edgeRDD)

In [0]:
%scala
// This block extracts the edges and vertices from the GraphX graph as DataFrames,
// and defines a sequence of top candidate vertex IDs for further analysis.

val edgesDF    = graph.edges.toDF
val verticesDF = graph.vertices.toDF
val topCandidateIds = Seq(1902943).map(_.toLong)

In [0]:
%scala

// Multi-source Dijkstra's algorithm using GraphX Pregel API.
// Computes shortest paths from multiple origins, tracking distance, origin, and path for each vertex.

def multiSourceDijkstra[VD](
    g: Graph[VD, Double], 
    origins: Seq[VertexId]
): Graph[(Double, VertexId, List[VertexId]), Double] = {  
  // Initialize vertices: (distance, origin, path)
  val initialGraph = g.mapVertices((id, _) =>
    if (origins.contains(id)) 
      (0.0, id, List(id))  // origin vertex: distance=0, origin=itself, path=[id]
    else 
      (Double.PositiveInfinity, -1L, List())    // others: infinity, no origin, empty path
  )

  val sssp = initialGraph.pregel((Double.PositiveInfinity, -1L, List[VertexId]()))(
    // Vertex program: keep the path with smallest distance
    (id, current, newUpdate) => 
      if (current._1 < newUpdate._1) current else newUpdate,

    // Send messages to neighbors if a shorter path is found
    triplet => {
      val newDist = triplet.srcAttr._1 + triplet.attr
      if (newDist < triplet.dstAttr._1) {
        Iterator((triplet.dstId, (newDist, triplet.srcAttr._2, triplet.srcAttr._3 :+ triplet.dstId)))
      } else {
        Iterator.empty
      }
    },

    // Merge messages: keep the one with the smallest distance
    (a, b) => if (a._1 < b._1) a else b
  )

  sssp
}

// Run Dijkstra's algorithm for all top candidate origins
val result = multiSourceDijkstra(graph.mapEdges(e => e.attr), topCandidateIds)

In [0]:
%scala
// This block converts the result GraphX vertices to a DataFrame with columns for start point, end point, distance, and path.
// It then orders the DataFrame by distance in descending order and displays the result.

val resultDF = result.vertices.map { case (id, (distance, origin, path)) =>
  (origin, id, distance, path.mkString(", ")) // (start, end, distance, path)
}.toDF("start_point", "end_point", "distance", "path")

// Order by distance in descending order (or ascending, if preferred)
val orderedResultDF = resultDF.orderBy($"distance".desc)

// Display the result DataFrame
display(orderedResultDF)

In [0]:
%scala
val catalogName = "geospatial"
val schema = "shortest_path"

val sqlQuery = s"""
  CREATE SCHEMA IF NOT EXISTS ${catalogName}.${schema}
  COMMENT 'This schema contains ${schema} data of the UK'
"""

spark.sql(sqlQuery)

In [0]:
%scala
// For each candidate start node, filter the shortest path results and save them as Delta tables in Unity Catalog.
// The table name includes the start node ID and the username for uniqueness.

topCandidateIds.foreach { startId =>
  val pathsForStartId = orderedResultDF.filter($"start_point" === startId)

  val tableName = s"start_point_$startId"

  pathsForStartId.write
    .format("delta")
    .mode("overwrite") // Overwrite existing table if present
    .option("overwriteSchema", "true") // Overwrite schema if it changes
    .saveAsTable(s"${catalogName}.${schema}.${tableName}_${username}")
}

In [0]:
%sql
SELECT max(distance) FROM geospatial.shortest_path.start_point_1902943